In [1]:
from railjson_generator import (
    InfraBuilder,
    SimulationBuilder,
    Location,
)
from railjson_generator.schema.infra.direction import Direction

from  railjson_generator.schema.infra.infra import Infra


In [2]:
builder = InfraBuilder()


In [3]:
T = [
    builder.add_track_section(label='T'+str(id), length=500)
    for id in range(6)
]

In [4]:
builder.add_link(T[2].end(), T[3].begin())
builder.add_point_switch(
    T[2].begin(),
    T[0].end(),
    T[1].end(),
    label='CVG',
)
builder.add_point_switch(
    T[3].end(),
    T[4].begin(),
    T[5].begin(),
    label='DVG',
)

PointSwitch(label='DVG', delay=0, base=TrackEndpoint(track_section=TrackSection(length=500, label='T3', track_name='placeholder_track', track_number=0, line_code=0, line_name='placeholder_line', waypoints=[], signals=[], operational_points=[], coordinates=[(None, None), (None, None)], slopes=[], curves=[]), endpoint=<Endpoint.END: 1>), left=TrackEndpoint(track_section=TrackSection(length=500, label='T4', track_name='placeholder_track', track_number=0, line_code=0, line_name='placeholder_line', waypoints=[], signals=[], operational_points=[], coordinates=[(None, None), (None, None)], slopes=[], curves=[]), endpoint=<Endpoint.BEGIN: 0>), right=TrackEndpoint(track_section=TrackSection(length=500, label='T5', track_name='placeholder_track', track_number=0, line_code=0, line_name='placeholder_line', waypoints=[], signals=[], operational_points=[], coordinates=[(None, None), (None, None)], slopes=[], curves=[]), endpoint=<Endpoint.BEGIN: 0>))

In [5]:
D = [
    T[i].add_detector(label='D'+str(i), position=450)
    if i in [0, 1, 3]
    else T[i].add_detector(label='D'+str(i), position=50)
    for i in range(6)
]

In [6]:
D

[Detector(position=450, applicable_direction=<ApplicableDirection.BOTH: 2>, label='D0'),
 Detector(position=450, applicable_direction=<ApplicableDirection.BOTH: 2>, label='D1'),
 Detector(position=50, applicable_direction=<ApplicableDirection.BOTH: 2>, label='D2'),
 Detector(position=450, applicable_direction=<ApplicableDirection.BOTH: 2>, label='D3'),
 Detector(position=50, applicable_direction=<ApplicableDirection.BOTH: 2>, label='D4'),
 Detector(position=50, applicable_direction=<ApplicableDirection.BOTH: 2>, label='D5')]

In [7]:
S = [
    T[i].add_signal(D[i].position-20, Direction.START_TO_STOP, D[i])
    for i, _ in enumerate(D)
]
LS = [
    signal.add_logical_signal("BAL", settings={"Nf": "true"})
    for signal in S
]

In [8]:
LS

[LogicalSignal(signaling_system='BAL', next_signaling_systems=[], settings={'Nf': 'true'}),
 LogicalSignal(signaling_system='BAL', next_signaling_systems=[], settings={'Nf': 'true'}),
 LogicalSignal(signaling_system='BAL', next_signaling_systems=[], settings={'Nf': 'true'}),
 LogicalSignal(signaling_system='BAL', next_signaling_systems=[], settings={'Nf': 'true'}),
 LogicalSignal(signaling_system='BAL', next_signaling_systems=[], settings={'Nf': 'true'}),
 LogicalSignal(signaling_system='BAL', next_signaling_systems=[], settings={'Nf': 'true'})]

In [9]:
infra = builder.build().to_rjs().dict()

In [10]:
import rlway.osrd.infra as infr

In [11]:
infr.routes(infra)

['rt.D0->D2',
 'rt.buffer_stop.0->D0',
 'rt.D1->D2',
 'rt.buffer_stop.1->D1',
 'rt.D2->D3',
 'rt.D3->D5',
 'rt.D3->D4',
 'rt.D4->buffer_stop.2',
 'rt.D5->buffer_stop.3']

In [12]:
infr.draw_infra(infra)